In [139]:
import pandas as pd
from spacy.lang.en import English
import itertools

In [140]:
post_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_post.csv', encoding='UTF-8')
comment_df = pd.read_csv('../predicting-satisfaction-using-graphs/csv/dataset/liwc_comment.csv', encoding='UTF-8')

In [141]:
satisfactions_float = list(post_df['satisfaction'])

In [143]:
nlp = English()
nlp.add_pipe("sentencizer")

In [146]:
post_contents = list(post_df['content'])
comment_bodies = list(comment_df['content'])

post_sequences = []
comment_sequences = []


for post_content, comment_body in zip(post_contents, comment_bodies):
    post_sequences.append(list(map(lambda x: str(x), list(nlp(post_content).sents))))
    comment_sequences.append(list(map(lambda x: str(x), list(nlp(comment_body).sents))))



# satisfaction score (y)
satisfactions_float = list(post_df['satisfaction'])
satisfactions = []
for_contrastive = []

for s in satisfactions_float:
    if s < 3.5:
        satisfactions.append(0)
    elif s < 5:
        satisfactions.append(1)
    else:
        satisfactions.append(2)
        
for s in satisfactions_float:
    if s <= 2.5:
        for_contrastive.append(0)
    elif s <= 3.5:
        for_contrastive.append(1)
    elif s <= 4.5:
        for_contrastive.append(2)
    elif s <= 5.5:
        for_contrastive.append(3)
    else:
        for_contrastive.append(4)

# print(satisfactions_float)
# print(satisfactions)

data = []

i = 0
for post, comment, satisfaction1, satisfaction2, satisfaction_float in zip(post_contents, comment_bodies,
                                                                           satisfactions, for_contrastive, satisfactions_float):
    data.append([i, post, comment, satisfaction1, satisfaction2, satisfaction_float])
    i += 1

columns = ['index', 'post_contents', 'comment_contents', 'label', 'label_for_contrastive', 'score']
df = pd.DataFrame(data, columns=columns)

In [147]:
comb_df = pd.DataFrame([], columns=columns)

In [148]:
comb_df

,index,post_contents,comment_contents,label,label_for_contrastive,score


In [149]:
for i in range(5):
    sub_df = df[df.label_for_contrastive == i].sample(50)
    comb_df = pd.concat([comb_df, sub_df])

In [150]:
satisfaction_comb = list(itertools.combinations(comb_df['index'], 2))

In [152]:
def make_comb_for_contrastive_learner(comb_list, df):
    columns = columns = ['index', 'post_contents', 'comment_contents', 'label', 'label_for_contrastive', 'score'] * 2
    result_df = pd.DataFrame([], columns=columns)
    for comb in comb_list:
        sub_comb_df = pd.concat([df[df.index == comb[0]].reset_index(drop=True), 
                                 df[df.index == comb[1]].reset_index(drop=True)], axis=1)
        result_df = pd.concat([result_df, sub_comb_df]).reset_index(drop=True)
    
    return result_df

result_df = make_comb_for_contrastive_learner(satisfaction_comb, df)

In [153]:
result_df.columns = ['index1', 'post_contents1', 'comment_contents1', 'label1', 'label_for_contrastive1', 'score1', 'index2', 'post_contents2', 'comment_contents2', 'label2', 'label_for_contrastive2', 'score2']

In [154]:
result_df.to_csv('../predicting-satisfaction-using-graphs/csv/dataset/df_for_contrastive_learner.csv')